<a href="https://colab.research.google.com/github/TuragDev/Predicting-Crystal-System-with-ML-DL/blob/main/Crystal_structure_prediction_by_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.2 MB/s eta 0:00:00


In [2]:
pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install scikit-learn

In [4]:
pip install keras

In [5]:
pip install tensorflow

In [6]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [7]:
pip install keras-tuner

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import sys
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.utils import shuffle
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import BayesianOptimization

<ipython-input-8-d5a2f462d84b>:46: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [10]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_test.csv to Orbital_test.csv
Saving Orbital_train.csv to Orbital_train.csv
Saving Sine_test.csv to Sine_test.csv
Saving Sine_train.csv to Sine_train.csv
Saving Xrd_test.csv to Xrd_test.csv
Saving Xrd_train.csv to Xrd_train.csv


In [ ]:
df= pd.read_csv('Sine_train.csv')
df.shape

(596, 166)

In [ ]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
591      monoclinic
592        trigonal
593    orthorhombic
594        trigonal
595      monoclinic
Name: crystal_system, Length: 596, dtype: object

In [ ]:
X_train.shape

(596, 152)

In [ ]:
df=pd.read_csv('Sine_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145         cubic
146      trigonal
147     hexagonal
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [ ]:
X_test.shape

(149, 152)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(152,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 12, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=2, max_value=152, step=3, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

In [ ]:
# Define the BayesianOptimization tuner
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,  # Number of random initializations
    max_trials=1000,  # Number of Bayesian optimization iterations
    directory='tuner_dir',  # Directory to store results
    project_name='classification_model'
)

# Perform hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compile the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model on the entire training dataset
history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

# Evaluate the model on the training and testing data
train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

# Print the results
print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Trial 1000 Complete [00h 00m 11s]
val_accuracy: 0.5166666507720947

Best val_accuracy So Far: 0.5916666388511658
Total elapsed time: 04h 13m 13s
Epoch 1/50
19/19 [==============================] - 1s 2ms/step - loss: 34.1014 - accuracy: 0.4732
Epoch 2/50
19/19 [==============================] - 0s 2ms/step - loss: 60.0277 - accuracy: 0.5302
Epoch 3/50
19/19 [==============================] - 0s 2ms/step - loss: 20.2241 - accuracy: 0.5503
Epoch 4/50
19/19 [==============================] - 0s 2ms/step - loss: 15.9275 - accuracy: 0.5772
Epoch 5/50
19/19 [==============================] - 0s 2ms/step - loss: 11.4210 - accuracy: 0.5973
Epoch 6/50
19/19 [==============================] - 0s 2ms/step - loss: 4.7240 - accuracy: 0.6326
Epoch 7/50
19/19 [==============================] - 0s 2ms/step - loss: 5.2128 - accuracy: 0.6158
Epoch 8/50
19/19 [==============================] - 0s 2ms/step - loss: 2.7245 - accuracy: 0.6242
Epoch 9/50
19/19 [==============================] - 0s 2ms/step - 

In [ ]:
df= pd.read_csv('Orbital_train.csv')
df.shape

(595, 1038)

In [ ]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
590    orthorhombic
591        trigonal
592      monoclinic
593       triclinic
594        trigonal
Name: crystal_system, Length: 595, dtype: object

In [ ]:
X_train.shape

(595, 1024)

In [ ]:
df=pd.read_csv('Orbital_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145      trigonal
146     hexagonal
147         cubic
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [ ]:
X_test.shape

(149, 1024)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(1024,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=32, max_value=1024, step=18, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

In [ ]:
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,
    max_trials=1200,
    directory='tuner_dir',
    project_name='classification_model'
)

# hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Building model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compilimg the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Trial 1200 Complete [00h 00m 12s]
val_accuracy: 0.5714285969734192

Best val_accuracy So Far: 0.605042040348053
Total elapsed time: 08h 31m 49s
Epoch 1/50
19/19 [==============================] - 1s 8ms/step - loss: 8.4028 - accuracy: 0.4941
Epoch 2/50
19/19 [==============================] - 0s 8ms/step - loss: 14.4752 - accuracy: 0.5193
Epoch 3/50
19/19 [==============================] - 0s 8ms/step - loss: 9.1021 - accuracy: 0.5782
Epoch 4/50
19/19 [==============================] - 0s 8ms/step - loss: 6.5627 - accuracy: 0.5580
Epoch 5/50
19/19 [==============================] - 0s 8ms/step - loss: 5.0825 - accuracy: 0.6202
Epoch 6/50
19/19 [==============================] - 0s 8ms/step - loss: 15.9220 - accuracy: 0.6017
Epoch 7/50
19/19 [==============================] - 0s 8ms/step - loss: 8.5273 - accuracy: 0.6168
Epoch 8/50
19/19 [==============================] - 0s 8ms/step - loss: 13.1690 - accuracy: 0.6168
Epoch 9/50
19/19 [==============================] - 0s 8ms/step - los

In [11]:
df= pd.read_csv('Xrd_train.csv')
df.shape

(596, 142)

In [12]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
591      monoclinic
592        trigonal
593    orthorhombic
594        trigonal
595      monoclinic
Name: crystal_system, Length: 596, dtype: object

In [13]:
X_train.shape

(596, 128)

In [14]:
df=pd.read_csv('Xrd_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0          cubic
1          cubic
2      triclinic
3      triclinic
4      triclinic
         ...    
144     trigonal
145        cubic
146     trigonal
147    hexagonal
148        cubic
Name: crystal_system, Length: 149, dtype: object

In [15]:
X_test.shape

(149, 128)

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [20]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization

def build_classification_model(hp):
    inputs = tf.keras.Input(shape=(128,))  # Update the input shape to match your dataset

    x = inputs

    # Add one or more dense layers with hyperparameter tuning
    for i in range(hp.Int("dense_layers", 1, 8, default=1)):
        x = tf.keras.layers.Dense(
            units=hp.Int('dense_units'+str(i), min_value=32, max_value=128, step=2, default=32),
            activation='relu',
        )(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.1, 0.8, step=0.1, default=0.3))(x)

    # Output layer for classification with 7 classes
    outputs = tf.keras.layers.Dense(7, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',  # Use appropriate classification loss
        metrics=['accuracy']  # Use accuracy for classification
    )

    return model

In [21]:
tuner = BayesianOptimization(
    build_classification_model,
    objective='val_accuracy',
    num_initial_points=5,  # Number of random initializations
    max_trials=800,  # Number of Bayesian optimization iterations
    directory='tuner_dir',  # Directory to store results
    project_name='classification_model'
)

# Perform hyperparameter tuning
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hparams = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = build_classification_model(best_hparams)

# Compile the model
best_model.compile(
    optimizer=keras.optimizers.Adam(best_hparams.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model on the entire training dataset
history = best_model.fit(X_train_normalized, y_train_encoded, epochs=50)

# Evaluate the model on the training and testing data
train_accuracy = best_model.evaluate(X_train_normalized, y_train_encoded, verbose=0)[1]
test_accuracy = best_model.evaluate(X_test_normalized, y_test_encoded, verbose=0)[1]

# Print the results
print("Best Hyperparameters:")
print(best_hparams.get_config())
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Trial 800 Complete [00h 00m 08s]
val_accuracy: 0.5416666865348816

Best val_accuracy So Far: 0.6000000238418579
Total elapsed time: 02h 13m 39s
Epoch 1/50
19/19 [==============================] - 1s 5ms/step - loss: 1.5965 - accuracy: 0.5638
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 1.1342 - accuracy: 0.6594
Epoch 3/50
19/19 [==============================] - 0s 4ms/step - loss: 0.9492 - accuracy: 0.6544
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 0.9380 - accuracy: 0.6628
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 0.8670 - accuracy: 0.6577
Epoch 6/50
19/19 [==============================] - 0s 3ms/step - loss: 0.8364 - accuracy: 0.6762
Epoch 7/50
19/19 [==============================] - 0s 4ms/step - loss: 0.8230 - accuracy: 0.6963
Epoch 8/50
19/19 [==============================] - 0s 5ms/step - loss: 0.7771 - accuracy: 0.7315
Epoch 9/50
19/19 [==============================] - 0s 6ms/step - loss: 